# Imports

In [ ]:
import os
import glob
import shutil
import pandas as pd

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!apt-get install lz4

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  lz4
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 90.0 kB of archives.
After this operation, 236 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 lz4 amd64 1.9.3-2build2 [90.0 kB]
Fetched 90.0 kB in 1s (68.5 kB/s)
Selecting previously unselected package lz4.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../lz4_1.9.3-2build2_amd64.deb ...
Unpacking lz4 (1.9.3-2build2) ...
Setting up lz4 (1.9.3-2build2) ...
Processing triggers for man-db (2.10.2-1) ...


# Setup

## Input and output paths definition and creation

In [ ]:
ddl_terra_out_tsv_path = "https://raw.githubusercontent.com/ImagingDataCommons/idc-prostate-mri-analysis/refs/heads/main/terra_mhub/data_tables/terra_mhub_prostatex_inference_only.tsv" #@param{type:"string"}
local_terra_out_path = "terra_mhub_prostatex_inference_only.tsv"
!wget -O {local_terra_out_path} {ddl_terra_out_tsv_path}

--2024-10-31 17:47:57--  https://raw.githubusercontent.com/ccosmin97/idc-prostate-mri-analysis/refs/heads/main/terra_mhub/data_tables/terra_mhub_prostatex_inference_only.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144229 (141K) [text/plain]
Saving to: ‘terra_mhub_prostatex_inference_only.tsv’

terra_mhub_prostate 100%[===================>] 140.85K  --.-KB/s    in 0.08s   

2024-10-31 17:47:57 (1.67 MB/s) - ‘terra_mhub_prostatex_inference_only.tsv’ saved [144229/144229]



In [ ]:
!rm -rf out_terra
#RAW ARCHIVE OUTPUTS
!mkdir -p out_terra/archives_final
#AI OUTPUTS
!mkdir -p out_terra/ai_dicom_seg
!mkdir -p out_terra/ai_dicom_sr

## Processing functions

In [ ]:
def create_batch_dir(base_dir, collection_id, batch_id):
  for out_dir in ["ai_dicom_seg", "ai_dicom_sr",
                  "archives_final"]:
    out_path = os.path.join(base_dir, out_dir, str(collection_id), str(batch_id))
    !rm -rf {out_path}
    !mkdir -p {out_path}

In [ ]:
def ddl_unzip_terra_archive(archive_lz4, output_folder):
  archive_filemame = archive_lz4.split("/")[-1]
  zip_ddl_path = os.path.join(output_folder)
  !gsutil -m cp -r {archive_lz4} {zip_ddl_path}
  # !cp $archive_lz4 $zip_ddl_path
  zip_path = os.path.join(output_folder, archive_filemame)
  !lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
  #move archive contents to output folder
  # return glob.glob(os.path.join(zip_ddl_path, "**", "*.dcm"), recursive=True)

## Load terra results data

In [ ]:
terra_out_df = pd.read_csv(local_terra_out_path, sep="\t")

In [ ]:
terra_out_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 17 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   entity:terra_mhub_prostatex_inference_only_id  36 non-null     int64 
 1   adcSeriesInstanceUIDs                          36 non-null     object
 2   collection_id                                  36 non-null     object
 3   dicomCodeMeaningProstate_lst                   36 non-null     object
 4   dicomCodeValuesProstate_lst                    36 non-null     object
 5   dicomCodingSchemeDesignatorProstate_lst        36 non-null     object
 6   dicomSrAiCodeMeaning_lst                       36 non-null     object
 7   dicomSrAiCodeValues_lst                        36 non-null     object
 8   dicomSrAiCodingSchemeDesignator_lst            36 non-null     object
 9   finalCompressedOutputFile                      36 non-null     obje

## Unzip terra archives

In [ ]:
for index, row in terra_out_df.iterrows():
  collection_id = row['collection_id']
  batch_id = row['entity:terra_mhub_prostatex_inference_only_id']
  seriesInstanceUIDs = row['seriesInstanceUIDs']
  finalCompressedOutputFile = row['finalCompressedOutputFile']
  print(f"batch id : {batch_id}")
  print(f"seriesInstanceUIDs :{seriesInstanceUIDs}")
  print(f"finalCompressedOutputFile :{finalCompressedOutputFile}")
  create_batch_dir("out_terra", collection_id, batch_id)
  #COPY and UNZIP final archive
  ddl_unzip_terra_archive(finalCompressedOutputFile,
                          os.path.join("out_terra", "archives_final",
                                       collection_id, str(batch_id)))
  #move AI DICOM SEG/SR objects to ai_dicom_seg/ai_dicom_sr
  ## AI DICOM SEG
  for object_list in  glob.glob(os.path.join("out_terra", "archives_final",
                                             str(collection_id), str(batch_id),
                                             "aggregated_results", "ai_results",
                                             "dicom_seg/*")):
    print(object_list)
    out_path = os.path.join("out_terra", "ai_dicom_seg", str(collection_id), str(batch_id))
    !cp -r {object_list} {out_path}
  ## AI DICOM SR
  for object_list in  glob.glob(os.path.join("out_terra", "archives_final",
                                             str(collection_id), str(batch_id),
                                             "aggregated_results", "ai_results",
                                             "dicom_sr/*")):
    out_path = os.path.join("out_terra", "ai_dicom_sr", str(collection_id), str(batch_id))
    !cp -r {object_list} {out_path}
  print("...")
  print("\n")

Streaming output truncated to the last 5000 lines.
aggregated_results/ai_results/dicom_seg/nnunet_prostate_task24/1.3.6.1.4.1.14519.5.2.1.7310.5101.170942338015901063429061763626/
aggregated_results/ai_results/dicom_seg/nnunet_prostate_task24/1.3.6.1.4.1.14519.5.2.1.7310.5101.170942338015901063429061763626/1.3.6.1.4.1.14519.5.2.1.7310.5101.170942338015901063429061763626_nnunet_mr_prostate.seg.dcm
aggregated_results/ai_results/dicom_seg/bamf_nnunet_mr_prostate/
aggregated_results/ai_results/dicom_seg/bamf_nnunet_mr_prostate/1.3.6.1.4.1.14519.5.2.1.7311.5101.249945594986863677107973338194/
aggregated_results/ai_results/dicom_seg/bamf_nnunet_mr_prostate/1.3.6.1.4.1.14519.5.2.1.7311.5101.249945594986863677107973338194/1.3.6.1.4.1.14519.5.2.1.7311.5101.249945594986863677107973338194_bamf_nnunet_mr_prostate.seg.dcm
aggregated_results/ai_results/dicom_seg/bamf_nnunet_mr_prostate/1.3.6.1.4.1.14519.5.2.1.7310.5101.303525861750852730995284862930/
aggregated_results/ai_results/dicom_seg/bamf_nnun

In [ ]:
!mv /content/out_terra/archives_final /content/

## Create sorted_data zip

In [ ]:
!zip -r results_sorted.zip /content/out_terra

Streaming output truncated to the last 5000 lines.
  adding: content/out_terra/ai_dicom_seg/prostatex_inference_only/31/bamf_nnunet_mr_prostate/1.3.6.1.4.1.14519.5.2.1.7311.5101.606554985338789812728613003926/1.3.6.1.4.1.14519.5.2.1.7311.5101.606554985338789812728613003926_bamf_nnunet_mr_prostate.seg.dcm (deflated 98%)
  adding: content/out_terra/ai_dicom_seg/prostatex_inference_only/31/bamf_nnunet_mr_prostate/1.3.6.1.4.1.14519.5.2.1.7310.5101.662798695442251537947237054156/ (stored 0%)
  adding: content/out_terra/ai_dicom_seg/prostatex_inference_only/31/bamf_nnunet_mr_prostate/1.3.6.1.4.1.14519.5.2.1.7310.5101.662798695442251537947237054156/1.3.6.1.4.1.14519.5.2.1.7310.5101.662798695442251537947237054156_bamf_nnunet_mr_prostate.seg.dcm (deflated 98%)
  adding: content/out_terra/ai_dicom_seg/prostatex_inference_only/31/bamf_nnunet_mr_prostate/1.3.6.1.4.1.14519.5.2.1.7311.5101.276959752243390681174198503432/ (stored 0%)
  adding: content/out_terra/ai_dicom_seg/prostatex_inference_only/3

# Create/Import data to GCS buckets -> DICOM Stores -> BQ tables

## GCP/GCS variables definition

In [ ]:
#@title Global parameters
project_id = "idc-sandbox-003" #@param{type:"string"}
location="us" #@param{type:"string"}

In [ ]:
#@title Buckets
parent_bucket = "gs://prostate_seg_terra_prostatex_inf_only_mhub" #@param{type:"string"}
ai_dicom_seg_bucket=f"gs://{parent_bucket}/ai_dicom_seg" #@param{type:"string"}
ai_dicom_sr_bucket=f"gs://{parent_bucket}/ai_dicom_sr" #@param{type:"string"}

In [ ]:
#@title DICOM STORES
dicom_dataset_id = "prostate_seg_terra_prostatex_inf_only_mhub_dataset" #@param{type:"string"}
ai_dicom_seg_store="aiDicomSegDicomStore" #@param{type:"string"}
ai_dicom_sr_store="aiDicomSrDicomStore" #@param{type:"string"}

In [ ]:
#@title BigQuery parameters
#@title test
bq_dataset_id = "prostate_seg_terra_mhub_v3_prostatex_inf_only" #@param{type:"string"}
bq_ai_dicom_seg = "ai_dicom_seg_table" #@param{type:"string"}
bq_ai_dicom_sr = "ai_dicom_sr_table" #@param{type:"string"}

## Set GCP project id

In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


## Create DICOM dataset

In [ ]:
!gcloud healthcare datasets delete $dicom_dataset_id --location=$location --project $project_id --quiet

Deleted dataset [prostate_seg_terra_prostatex_inf_only_mhub_dataset].


In [ ]:
!gcloud healthcare datasets create $dicom_dataset_id --location=$location --project $project_id

Create request issued for: [prostate_seg_terra_prostatex_inf_only_mhub_dataset]
Created dataset [prostate_seg_terra_prostatex_inf_only_mhub_dataset].


## Create DICOM Stores

In [ ]:
lst_dicom_stores = [ai_dicom_seg_store,
                    ai_dicom_sr_store]
for dicom_store in lst_dicom_stores:
  !gcloud healthcare dicom-stores delete $dicom_store \
    --dataset=$dicom_dataset_id \
    --location=$location \
    --project=$project_id --quiet
  !gcloud healthcare dicom-stores create $dicom_store \
    --dataset=$dicom_dataset_id \
    --location=$location \
    --project=$project_id --quiet

ERROR: (gcloud.healthcare.dicom-stores.delete) NOT_FOUND: DICOM store: projects/idc-sandbox-003/locations/us/datasets/prostate_seg_terra_prostatex_inf_only_mhub_dataset/dicomStores/aiDicomSegDicomStore not found. This command is authenticated as cciausu97@gmail.com which is the active account specified by the [core/account] property
Created dicomStore [aiDicomSegDicomStore].
ERROR: (gcloud.healthcare.dicom-stores.delete) NOT_FOUND: DICOM store: projects/idc-sandbox-003/locations/us/datasets/prostate_seg_terra_prostatex_inf_only_mhub_dataset/dicomStores/aiDicomSrDicomStore not found. This command is authenticated as cciausu97@gmail.com which is the active account specified by the [core/account] property
Created dicomStore [aiDicomSrDicomStore].


## Create parent GCS bucket

In [ ]:
!gcloud storage rm -r {parent_bucket} --project $project_id
!gcloud storage buckets delete {parent_bucket} --project $project_id

Removing objects:
Removing gs://prostate_seg_terra_prostatex_inf_only_mhub/ai_dicom_seg/prostatex_inference_only/0/bamf_nnunet_mr_prostate/1.3.6.1.4.1.14519.5.2.1.7310.5101.233894782007809884236855995183/1.3.6.1.4.1.14519.5.2.1.7310.5101.233894782007809884236855995183_bamf_nnunet_mr_prostate.seg.dcm#1729633781336541...
Removing gs://prostate_seg_terra_prostatex_inf_only_mhub/ai_dicom_seg/prostatex_inference_only/0/bamf_nnunet_mr_prostate/1.3.6.1.4.1.14519.5.2.1.7310.5101.337896560664468718350275750005/1.3.6.1.4.1.14519.5.2.1.7310.5101.337896560664468718350275750005_bamf_nnunet_mr_prostate.seg.dcm#1729633781835505...
Removing gs://prostate_seg_terra_prostatex_inf_only_mhub/ai_dicom_seg/prostatex_inference_only/0/bamf_nnunet_mr_prostate/1.3.6.1.4.1.14519.5.2.1.7311.5101.175570448971296597162147241386/1.3.6.1.4.1.14519.5.2.1.7311.5101.175570448971296597162147241386_bamf_nnunet_mr_prostate.seg.dcm#1729633782137822...
Removing gs://prostate_seg_terra_prostatex_inf_only_mhub/ai_dicom_seg/pro

In [ ]:
!gcloud storage buckets create {parent_bucket} --location=$location --project $project_id

Creating gs://prostate_seg_terra_prostatex_inf_only_mhub/...


## Move local data to GCS buckets

In [ ]:
#Move AI DICOM SEG
!gsutil -m cp -r /content/out_terra/ai_dicom_seg {parent_bucket}
#Move AI DICOM SR
!gsutil -m cp -r /content/out_terra/ai_dicom_sr {parent_bucket}

Copying file:///content/out_terra/ai_dicom_seg/prostatex_inference_only/5/monai_prostate158/1.3.6.1.4.1.14519.5.2.1.7311.5101.166265209513832781969059787909/1.3.6.1.4.1.14519.5.2.1.7311.5101.166265209513832781969059787909_monai_prostate158.seg.dcm [Content-Type=application/dicom]...
Copying file:///content/out_terra/ai_dicom_seg/prostatex_inference_only/5/monai_prostate158/1.3.6.1.4.1.14519.5.2.1.7311.5101.396250088865668447959433809122/1.3.6.1.4.1.14519.5.2.1.7311.5101.396250088865668447959433809122_monai_prostate158.seg.dcm [Content-Type=application/dicom]...
Copying file:///content/out_terra/ai_dicom_seg/prostatex_inference_only/5/monai_prostate158/1.3.6.1.4.1.14519.5.2.1.7310.5101.263194549315074328788933721372/1.3.6.1.4.1.14519.5.2.1.7310.5101.263194549315074328788933721372_monai_prostate158.seg.dcm [Content-Type=application/dicom]...
Copying file:///content/out_terra/ai_dicom_seg/prostatex_inference_only/5/monai_prostate158/1.3.6.1.4.1.14519.5.2.1.7311.5101.4729006849975092319294

## Export GCS bucket data to dicom stores

In [ ]:
!gcloud healthcare dicom-stores delete dicomAllAIIdcSegSr \
  --dataset=$dicom_dataset_id \
  --location=$location \
  --project=$project_id --quiet

!gcloud healthcare dicom-stores create dicomAllAIIdcSegSr \
  --dataset=$dicom_dataset_id \
  --location=$location \
  --project=$project_id --quiet

ERROR: (gcloud.healthcare.dicom-stores.delete) NOT_FOUND: DICOM store: projects/idc-sandbox-003/locations/us/datasets/prostate_seg_terra_prostatex_inf_only_mhub_dataset/dicomStores/dicomAllAIIdcSegSr not found. This command is authenticated as cciausu97@gmail.com which is the active account specified by the [core/account] property
Created dicomStore [dicomAllAIIdcSegSr].


In [ ]:
#ai dicom seg
temp_gcs_uri = f'{parent_bucket}/ai_dicom_seg/**.dcm'
!gcloud healthcare dicom-stores import gcs dicomAllAIIdcSegSr \
  --dataset=$dicom_dataset_id \
  --location=$location \
  --gcs-uri=$temp_gcs_uri \
  --project $project_id
#ai dicom sr
temp_gcs_uri = f'{parent_bucket}/ai_dicom_sr/**.dcm'
!gcloud healthcare dicom-stores import gcs dicomAllAIIdcSegSr \
  --dataset=$dicom_dataset_id \
  --location=$location \
  --gcs-uri=$temp_gcs_uri \
  --project $project_id

Request issued for: [dicomAllAIIdcSegSr]
name: projects/idc-sandbox-003/locations/us/datasets/prostate_seg_terra_prostatex_inf_only_mhub_dataset/dicomStores/dicomAllAIIdcSegSr
Request issued for: [dicomAllAIIdcSegSr]
name: projects/idc-sandbox-003/locations/us/datasets/prostate_seg_terra_prostatex_inf_only_mhub_dataset/dicomStores/dicomAllAIIdcSegSr


AI DICOM SEG

In [ ]:
temp_gcs_uri = f'{parent_bucket}/ai_dicom_seg/**.dcm'
!gcloud healthcare dicom-stores import gcs $ai_dicom_seg_store \
  --dataset=$dicom_dataset_id \
  --location=$location \
  --gcs-uri=$temp_gcs_uri \
  --project $project_id

Request issued for: [aiDicomSegDicomStore]
name: projects/idc-sandbox-003/locations/us/datasets/prostate_seg_terra_prostatex_inf_only_mhub_dataset/dicomStores/aiDicomSegDicomStore


AI DICOM SR

In [ ]:
temp_gcs_uri = f'{parent_bucket}/ai_dicom_sr/**.dcm'
!gcloud healthcare dicom-stores import gcs $ai_dicom_sr_store \
  --dataset=$dicom_dataset_id \
  --location=$location \
  --gcs-uri=$temp_gcs_uri \
  --project $project_id

Request issued for: [aiDicomSrDicomStore]
name: projects/idc-sandbox-003/locations/us/datasets/prostate_seg_terra_prostatex_inf_only_mhub_dataset/dicomStores/aiDicomSrDicomStore


## Export DICOM Stores to BigQuery

## Create BigQuery dataset

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)
client.delete_dataset(
    bq_dataset_id, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(bq_dataset_id))

Deleted dataset 'prostate_seg_terra_mhub_v3_prostatex_inf_only'.


In [ ]:
!bq --location=$location mk -d \
--description "dataset for terra_mhub prostate segmentation analysis" \
--project_id=$project_id \
$bq_dataset_id

Dataset 'idc-sandbox-003:prostate_seg_terra_mhub_v3_prostatex_inf_only' successfully created.


Export terra output data table to bigquery

In [ ]:
from ast import literal_eval
import numpy as np
from google.cloud import bigquery
tempDf = pd.read_csv(local_terra_out_path, index_col=None, sep="\t")
col_lsts = ['adcSeriesInstanceUIDs',
            # 'collection_id',
            'dicomCodeMeaningProstate_lst',
            'dicomCodeValuesProstate_lst',
            'dicomCodingSchemeDesignatorProstate_lst',
            'dicomSrAiCodeMeaning_lst',
            'dicomSrAiCodeValues_lst',
            'dicomSrAiCodingSchemeDesignator_lst',
            # 'finalCompressedOutputFile',
            # 'mhubCompressedOutputFile',
            'mhub_model_name_lst',
            'mhubaiCustomSegmentAlgorithmName_lst',
            'mhubai_custom_config_lst',
            # 'radsAiCompressedOutputFile',
            # 'res_scheme_format',
            'seriesInstanceUIDs']
outTempDf = tempDf.copy()
for col_processed in outTempDf.columns.values:
  if col_processed in col_lsts:
    print(f"col_processed : {col_processed}")
    outTempDf[col_processed] = outTempDf[col_processed].apply(lambda x: literal_eval(x) if x is not np.nan else np.nan)
client = bigquery.Client(project="idc-sandbox-003")
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("entity:terra_mhub_prostatex_inference_only_id", bigquery.enums.SqlTypeNames.INT64), # create each column in Big Query along with types
        bigquery.SchemaField("adcSeriesInstanceUIDs", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("collection_id", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("dicomCodeMeaningProstate_lst", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("dicomCodeValuesProstate_lst", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("dicomCodingSchemeDesignatorProstate_lst", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("dicomSrAiCodeMeaning_lst", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("dicomSrAiCodeValues_lst", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("dicomSrAiCodingSchemeDesignator_lst", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("finalCompressedOutputFile", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("mhubCompressedOutputFile", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("mhub_model_name_lst", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("mhubaiCustomSegmentAlgorithmName_lst", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("mhubai_custom_config_lst", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("radsAiCompressedOutputFile", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
        bigquery.SchemaField("res_scheme_format", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("seriesInstanceUIDs", bigquery.enums.SqlTypeNames.STRING, mode='REPEATED'),
    ],
    write_disposition="WRITE_APPEND",
)
job = client.load_table_from_dataframe(
    outTempDf, f'idc-sandbox-003.{bq_dataset_id}.terra_mhub_prosatatex_inf_out_table', job_config=job_config)

col_processed : adcSeriesInstanceUIDs
col_processed : dicomCodeMeaningProstate_lst
col_processed : dicomCodeValuesProstate_lst
col_processed : dicomCodingSchemeDesignatorProstate_lst
col_processed : dicomSrAiCodeMeaning_lst
col_processed : dicomSrAiCodeValues_lst
col_processed : dicomSrAiCodingSchemeDesignator_lst
col_processed : mhub_model_name_lst
col_processed : mhubaiCustomSegmentAlgorithmName_lst
col_processed : mhubai_custom_config_lst
col_processed : seriesInstanceUIDs


## Export DICOM Stores to BigQuery tables

AI DICOM SEG

In [ ]:
bq_table_path = ".".join((project_id, bq_dataset_id, bq_ai_dicom_seg))
!gcloud healthcare dicom-stores export bq $ai_dicom_seg_store \
  --location=$location \
  --dataset=$dicom_dataset_id \
  --bq-table=bq://{bq_table_path}

Request issued for: [aiDicomSegDicomStore]
name: projects/idc-sandbox-003/locations/us/datasets/prostate_seg_terra_prostatex_inf_only_mhub_dataset/dicomStores/aiDicomSegDicomStore


AI DICOM SR

In [ ]:
bq_table_path = ".".join((project_id, bq_dataset_id, bq_ai_dicom_sr))
!gcloud healthcare dicom-stores export bq $ai_dicom_sr_store \
  --location=$location \
  --dataset=$dicom_dataset_id \
  --bq-table=bq://{bq_table_path}

Request issued for: [aiDicomSrDicomStore]
name: projects/idc-sandbox-003/locations/us/datasets/prostate_seg_terra_prostatex_inf_only_mhub_dataset/dicomStores/aiDicomSrDicomStore
